# Prime Python API Overview

## Outline

1. Configuring credentials
2. Retrieving objects
   - Loss Sets
   - Layers
   - Portfolios
   - Event Catalogs
   - Analysis Profiles
3. Uploading data
   - Loss Sets
4. Creating new objects
   - Layers
   - Portfolios
5. Modifying objects
   - Modify a layer
   - Modify a portfolio
6. Creating views
   - Layer Views
   - Portfolio Views
7. Querying metrics
   - Layer metrics
   - Portfolio metrics
8. Performance optimization
9. Advanced metrics
   - Co-Metrics
   - Window Metrics

## Configuring Credentials

Before getting started, you will need to access your server. You can do this by assigning your server's URL to `analyzere.base_url`. You can also replace your username. Instead of hard coding the password, we recommend that you use the prompt `getpass()`.

In [ ]:
import analyzere
from getpass import getpass

In [ ]:
analyzere.base_url = "[insert your server's URL here]"

In [ ]:
analyzere.username = "analyzere"

In [ ]:
analyzere.password = getpass()

 ········


## Retrieving objects

### Loss Sets

In [ ]:
from analyzere import LossSet

#### Listing loss sets

In [ ]:
ls_from_list = LossSet.list()

In [ ]:
ls_from_list[0]

<LossSet id=09c8a537-95e1-4fde-a943-317d4bd90d6f at 0x7f4968245160> JSON: {
  "_type": "YELTLossSet",
  "adjust_leap_years": true,
  "created": "2022-04-12T19:26:06.914535+00:00",
  "currency": "USD",
  "data": {
    "ref_id": "1a4cd9e6-2ffd-4fc4-adca-515f463a8f28"
  },
  "data_type": "csv",
  "description": "Commercial_B - EU",
  "event_catalogs": [
    {
      "ref_id": "8d9a565c-e8bf-43ea-bc06-999f40dba547"
    }
  ],
  "id": "09c8a537-95e1-4fde-a943-317d4bd90d6f",
  "loss_type": "LossGross",
  "meta_data": {
    "line_of_business": "Commercial_B",
    "md5": "dba98aa58aeca0e25f052fba51107c21",
    "peril": "NonModelled",
    "program_id": "10001",
    "region": "Europe",
    "vendor_name": "NonModelled",
    "vendor_version": "0.0"
  },
  "modified": "2022-04-12T19:26:08.354578+00:00",
  "one_based_sequencing": true,
  "profile": {
    "attributes": {
      "LocationILFraction": [
        "1.0"
      ],
      "ModelTypeName": [
        "NonModelled"
      ],
      "PerilLocationID"

#### Retrive a single loss set

In [ ]:
ls_from_id = LossSet.retrieve(ls_from_list[0].id)

In [ ]:
ls_from_id

<LossSet id=09c8a537-95e1-4fde-a943-317d4bd90d6f at 0x7f4958760d30> JSON: {
  "_type": "YELTLossSet",
  "adjust_leap_years": true,
  "created": "2022-04-12T19:26:06.914535+00:00",
  "currency": "USD",
  "data": {
    "ref_id": "1a4cd9e6-2ffd-4fc4-adca-515f463a8f28"
  },
  "data_type": "csv",
  "description": "Commercial_B - EU",
  "event_catalogs": [
    {
      "ref_id": "8d9a565c-e8bf-43ea-bc06-999f40dba547"
    }
  ],
  "id": "09c8a537-95e1-4fde-a943-317d4bd90d6f",
  "loss_type": "LossGross",
  "meta_data": {
    "line_of_business": "Commercial_B",
    "md5": "dba98aa58aeca0e25f052fba51107c21",
    "peril": "NonModelled",
    "program_id": "10001",
    "region": "Europe",
    "vendor_name": "NonModelled",
    "vendor_version": "0.0"
  },
  "modified": "2022-04-12T19:26:08.354578+00:00",
  "one_based_sequencing": true,
  "profile": {
    "attributes": {
      "LocationILFraction": [
        "1.0"
      ],
      "ModelTypeName": [
        "NonModelled"
      ],
      "PerilLocationID"

#### Search loss sets

In [ ]:
ls_from_search = LossSet.list(search="74463-34-1")

In [ ]:
[ (ls.id, ls.description) for ls in ls_from_search]

[('ca21052a-9872-4cca-ab07-556fab39b410', '74463-34-1'),
 ('00041535-988c-4d8a-80bc-55612c047d22', '74463-34-1'),
 ('cec1b80d-ef2a-45de-87ad-1e8fb59bd599', '74463-34-1'),
 ('ea648106-14d9-4c9f-a5d8-e68e6341e3be', '74463-34-1'),
 ('36c319fe-a3f3-4992-ab3b-95c0f58905a7', '74463-34-1')]

### Layers

In [ ]:
from analyzere import Layer

#### Listing layers

In [ ]:
l_from_list = Layer.list()

In [ ]:
[ (l.id, l.description) for l in l_from_list[:10]]

[('cc23e019-e04a-4b39-8bb3-69f8c86e1381', 'Commercial_A - JPWS'),
 ('85b313aa-e96a-4468-ae10-5a3df69d3987', 'Commercial_B - NAHU'),
 ('08fab36d-5e5b-420a-b653-0944b459256e', 'Commercial_A - JPEQ'),
 ('4377cd54-821d-4208-bdb9-0ea56a706701', 'Residential - NAHU'),
 ('55e11a62-e5eb-45b2-95c5-9b33820cc0b6', 'Residential - USWF'),
 ('fc53d466-7bc8-473a-9189-e7d5fc452233', 'Commercial_B - USEQ'),
 ('11ed1b72-3d01-4a72-abb4-d0e8d025026c', 'Residential - USEQ'),
 ('e4e1978e-b89a-416f-924c-dd87519311d2', 'Commercial_A - JPEQ'),
 ('8a167578-0724-4322-984c-f88a53ad5257', 'Commercial_B - NAHU'),
 ('79e74707-c113-46be-838a-1ad423f56025', 'Commercial_A - JPWS')]

#### Retrieve a single layer

In [ ]:
l_from_id = Layer.retrieve(l_from_list[0].id)

In [ ]:
l_from_id

<Layer id=cc23e019-e04a-4b39-8bb3-69f8c86e1381 at 0x7f9b94220fd0> JSON: {
  "_type": "Generic",
  "aggregate_attachment": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 0.0,
    "value_date": null
  },
  "aggregate_limit": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 1.7976931348623157e+308,
    "value_date": null
  },
  "attachment": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 0.0,
    "value_date": null
  },
  "created": "2022-04-12T20:12:40.291617+00:00",
  "description": "Commercial_A - JPWS",
  "expiry_date": null,
  "fees": [],
  "franchise": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 0.0,
    "value_date": null
  },
  "id": "cc23e019-e04a-4b39-8bb3-69f8c86e1381",
  "inception_date": null,
  "limit": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 1.7976931348623157e+308,
    "value_date": null


#### Search layers

In [ ]:
l_from_search = Layer.list(search="74463-34-1")

In [ ]:
[ (l.id, l.description, l.meta_data["AnalysisID"]) for l in l_from_search]

[('60fff84a-8a83-490f-afce-01fe6b8e174d', '74463-34-1_2748451', '2748451'),
 ('3d488186-7304-4e76-ae70-6cf5b3ea042a', '74463-34-1_2748451', '2748451')]

#### Search by metadata fields

##### Search by business class

In [ ]:
l_from_metadata = Layer.list(metaquery='meta_data["BPU BC L3"]="CRPXU"')

In [ ]:
l_from_metadata.meta.total_count

14339

##### Search by program ID

In [ ]:
l_from_program_metadata = Layer.list(metaquery='meta_data["ProgramID"]="50640"')

In [ ]:
l_from_program_metadata.meta.total_count

8

##### Complex metadata queries

In [ ]:
l_from_complex_metadata = Layer.list(metaquery='(meta_data["BPU BC L3"]="CRPXU" AND meta_data["BPU Legal Entity"]="S0033") OR (meta_data["BPU BC L3"]="CRPXI" AND meta_data["BPU Legal Entity"]="HIBDA")')

In [ ]:
l_from_complex_metadata.meta.total_count

10368

#### Other list parameters

##### Limit the fields returned

In [ ]:
l_limited_fields = Layer.list(fields="id,description")

In [ ]:
l_limited_fields[0]

<Layer id=cc23e019-e04a-4b39-8bb3-69f8c86e1381 at 0x7f49585f99a0> JSON: {
  "description": "Commercial_A - JPWS",
  "id": "cc23e019-e04a-4b39-8bb3-69f8c86e1381"
}

##### Limit the number of results returned

In [ ]:
l_limited_responses = Layer.list(limit=10)

In [ ]:
len(l_limited_responses)

10

In [ ]:
l_limited_responses.meta.total_count

22075

##### Skip a number of results returned

In [ ]:
l_offset = Layer.list(offset=10, limit=10)

In [ ]:
len(l_offset)

10

In [ ]:
l_offset.meta.total_count

22075

##### Sort responses

Order responses by descending creation date/time.

In [ ]:
l_created_desc = Layer.list(ordering="-created")

Order responses by ascending modification date/time.

In [ ]:
l_modified_asc = Layer.list(ordering="modified")

### Portfolios

In [ ]:
from analyzere import Portfolio

#### List portfolios

In [ ]:
portfolios = Portfolio.list()

In [ ]:
portfolios.meta.total_count

33

#### Retrieve single portfolio

In [ ]:
portfolio = Portfolio.retrieve(portfolios[0].id)

In [ ]:
portfolio

<Portfolio id=ab4a6e11-d659-41ad-b1ec-6daa0d382928 at 0x7f9b824f1fa0> JSON: {
  "_type": "StaticPortfolio",
  "created": "2022-04-14T18:11:04.102552+00:00",
  "description": "Large Loss Portfolio",
  "id": "ab4a6e11-d659-41ad-b1ec-6daa0d382928",
  "layers": [
    {
      "ref_id": "51fd366a-de94-4de9-84d7-cc2cff9de1f2"
    },
    {
      "ref_id": "db6bbf66-28f6-4b37-a5ba-728ecebad76d"
    },
    {
      "ref_id": "dbc8a807-635e-4044-93d9-0c5c171eff9a"
    },
    {
      "ref_id": "cc85547a-c3fc-44c4-9afb-ad19375f8abd"
    },
    {
      "ref_id": "934b9558-53ec-4a35-80fe-444366f33e1e"
    },
    {
      "ref_id": "ebd5cb1e-5b69-43dc-805b-955adc5116ba"
    }
  ],
  "meta_data": {},
  "modified": "2022-04-14T18:11:04.102552+00:00",
  "name": "Large Loss Portfolio"
}

### Event Catalog

In [ ]:
from analyzere import EventCatalog

#### List event catalogs

In [ ]:
catalogs = EventCatalog.list()

In [ ]:
catalogs.meta.total_count

2

#### Retrieve single event catalog

In [ ]:
catalog = EventCatalog.retrieve(catalogs[0].id)

In [ ]:
catalog

<EventCatalog id=8d9a565c-e8bf-43ea-bc06-999f40dba547 at 0x7f9b826582b0> JSON: {
  "created": "2022-04-12T15:35:38.555044+00:00",
  "data": {
    "ref_id": "8423be69-18fd-4a45-bf1a-8c52d771568c"
  },
  "description": "Combined AIR v9 + RMS v21 + Non Modelled Catalog",
  "id": "8d9a565c-e8bf-43ea-bc06-999f40dba547",
  "meta_data": {},
  "modified": "2022-04-12T15:39:23.109153+00:00",
  "source": "AIR+RMS",
  "status": "processing_succeeded",
  "status_message": null,
  "tags": []
}

### Analysis Profile

In [ ]:
from analyzere import AnalysisProfile

#### List analysis profiles

In [ ]:
analysis_profiles = AnalysisProfile.list()

In [ ]:
analysis_profiles.meta.total_count

5

### Automatic Dereferencing

As you can see above, the Prime API makes ample use of cross-referencing objects that alreay exist on the server. The Python-bindings can automatically dereference those references and "lazy-load" objects from the server when they are being access.

For example, we will retrieve a Layer and then implicitly dereference and retrieve the associated Loss Set object and, in turn, the Loss Set's Event Catalog object. Note, the retrieval of those objects only includes their header data and not any underlying large datasets such as loss set data or event catalog data.

Also note, the dereferencing occurs implicitly on a per-access basis, which is not always the most efficient approach as it is inherently sequential and unknowingly to the user/programmer may incur unnecessary requests.

In [ ]:
Layer.list(search="Program 51015", limit=1)[0].loss_sets[0].event_catalogs[0].description

'Combined AIR v9 + RMS v21 + Non Modelled Catalog'

## Uploading data

Create a new loss set object and upload data associated with that newly created loss set object. However, first we need an event catalog as each loss set must be associated with an event catalog.

In [ ]:
ec = EventCatalog.retrieve('8d9a565c-e8bf-43ea-bc06-999f40dba547')
# OR search for an event catalog by name (searches always return lists)
ec = EventCatalog.list(search="Combined AIR v9 + RMS v21 + Non Modelled Catalog", limit=1)[0]

Now we can create the loss set object.

In [ ]:
ls = LossSet(
    # We always must specify the type of loss set. In this case we will upload a YELT loss set (Trial/Year, Time/Sequence, EventId, Loss)
    type="YELTLossSet",
    # We also must specify an event catalog. We use the one we retrieved above.
    event_catalogs=[ec],
    # A description is required and always good to be meaningfully selected with respect to the client-application.
    description="Test Loss Set for Python Training",
    # We also must specify the currency in which losses in the loss set are stored.
    currency="USD",
    # For YELTLossSets we must set a start date of where the simulation begins. You can also use `datetime` here.
    start_date="2022-01-01T00:00:00Z",
    # For YELTLossSets we must specify how many trials / years are represented by the loss set.
    trial_count=10000,
)    

This only created an object client-side. Now we need to save it to the server.

In [ ]:
ls.save()

<LossSet id=5228a1eb-0546-4768-95c1-cac600202a53 at 0x7f9b836079d0> JSON: {
  "_type": "YELTLossSet",
  "adjust_leap_years": false,
  "created": "2022-09-05T21:39:45.249112+00:00",
  "currency": "USD",
  "data": null,
  "data_type": "csv",
  "description": "Test Loss Set for Python Training",
  "event_catalogs": [
    {
      "ref_id": "8d9a565c-e8bf-43ea-bc06-999f40dba547"
    }
  ],
  "id": "5228a1eb-0546-4768-95c1-cac600202a53",
  "loss_type": "LossGross",
  "meta_data": {},
  "modified": "2022-09-05T21:39:45.249112+00:00",
  "one_based_sequencing": false,
  "profile": {
    "attributes": {},
    "avg_annual_loss": null,
    "currency": "USD",
    "max_loss": null,
    "min_loss": null,
    "non_zero_losses": null,
    "num_losses": null
  },
  "reinstatements_data_type": "percentage",
  "start_date": "2022-01-01T00:00:00+00:00",
  "status": "ready",
  "status_message": null,
  "trial_count": 10000
}

Status `ready` means that the object is ready to receive data.

In [ ]:
ls.status

'ready'

Now we upload the data in CSV format. `upload_data` blocks until the status is set to "Processing Successful" or "Processing Failed".

In [ ]:
ls.upload_data(open("AIR_Residential_NA_HU_USD_PRIME.csv", "rb"))

<EmbeddedResource at 0x7f9b82e842e0> JSON: {
  "bytes_uploaded": 2628144,
  "commit_progress": 100.0,
  "status": "Processing Successful",
  "total_bytes": 2628144
}

The loss set is now ready to be used.

## Creating Objects

### Create a Layer

Layers are all created in a similar manner: A Python `Layer` object is created with arguments that match the required JSON attributes. Here we will create a CatXL layer and use the loss set uploaded above as the subject losses to this layer. However, we must first import additional types from the library.

In [ ]:
from analyzere import MonetaryUnit, Reinstatement

In [ ]:
layer = Layer(
    # Type of layer: CatXL
    type="CatXL",
    # Loss sets associated with the layer
    loss_sets=[ls],
    # 3M USD attachment
    attachment=MonetaryUnit(3e6, "USD"),
    # 10M USD limit
    limit=MonetaryUnit(10e6, "USD"),
    # Franchise deductible is a required field, but we set it to 0 here.
    franchise=MonetaryUnit(0, "USD"),
    # The reinsurer's participation: 25%
    participation=0.25,
    # Inception date (optional, inclusive, supports `datetime`)
    inception_date="2022-01-01T00:00:00Z",
    # Expiry date (optional, exclusive, supports `datetime`)
    expiry_date="2023-01-01T00:00:00Z",
    # Description (optional)
    description="Test Layer for Python Training",
    # Additional metadata fields (optional)
    meta_data={
        "Test-LayerId": 12345,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True,
    }
).save()

In [ ]:
layer

<Layer id=7f825da0-d02b-4494-9fae-853fcfc7a328 at 0x7f9b81f0edc0> JSON: {
  "_type": "CatXL",
  "attachment": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 3000000.0,
    "value_date": null
  },
  "created": "2022-09-05T23:20:01.212052+00:00",
  "description": "Test Layer for Python Training",
  "expiry_date": "2023-01-01T00:00:00+00:00",
  "fees": [],
  "franchise": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 0.0,
    "value_date": null
  },
  "id": "7f825da0-d02b-4494-9fae-853fcfc7a328",
  "inception_date": "2022-01-01T00:00:00+00:00",
  "limit": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 10000000.0,
    "value_date": null
  },
  "loss_sets": [
    {
      "ref_id": "5228a1eb-0546-4768-95c1-cac600202a53"
    }
  ],
  "meta_data": {
    "Test-Latest": true,
    "Test-LayerId": 12345,
    "Test-Purpose": "Training",
    "Test-Version": 1
  },
  "modified": "2022-09-05T23:

### Create a Portfolio

In [ ]:
from analyzere import Portfolio

A portfolio is simply a collection of layers. In this example, we will create a portfolio of only one layer.

In [ ]:
portfolio = Portfolio(
    # A list of layers that are to me included in the portfolio. This can also be done
    layers=[layer],
    name="Test Portfolio for Python Training",
    description="Test Portfolio for Python Training",
    meta_data={
        "Test-PortfolioId": 12345,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True
    }
).save()

In [ ]:
portfolio

<Portfolio id=5714a745-4809-4665-86d2-29f1ebba1d91 at 0x7f9b81f0e280> JSON: {
  "_type": "StaticPortfolio",
  "created": "2022-09-05T23:38:22.122976+00:00",
  "description": "Test Portfolio for Python Training",
  "id": "5714a745-4809-4665-86d2-29f1ebba1d91",
  "layers": [
    {
      "ref_id": "7f825da0-d02b-4494-9fae-853fcfc7a328"
    }
  ],
  "meta_data": {
    "Test-Latest": true,
    "Test-PortfolioId": 12345,
    "Test-Purpose": "Training",
    "Test-Version": 1
  },
  "modified": "2022-09-05T23:38:22.122976+00:00",
  "name": "Test Portfolio for Python Training"
}

## Modifying Objects

Certain API objects such as Layers and Portfolios are mutable and can be modified after their initial creation. When these objects are modified they will retain their originally assigned ID. The Prime platform does not provide any built-in mechanisms for versioning or change history such that the original information stored with the object before modification is lost. However, there are client-side patterns that can be employed to store multiple versions of an object.

In the following example, we will update the layer we've created above with additional premium and reinstatement information. In addition, we will be creating a second layer which we will add to the previously created portfolio.

In [ ]:
# Retrieve originally created layer
layer = Layer.retrieve("7f825da0-d02b-4494-9fae-853fcfc7a328")

In [ ]:
# Update layer attributes
layer.premium = MonetaryUnit(1e6, "USD")
layer.reinstatements = [ 
    Reinstatement(premium=0.5, brokerage=0.1), 
    Reinstatement(premium=0.2, brokerage=0.1) 
]
layer.save()

<Layer id=7f825da0-d02b-4494-9fae-853fcfc7a328 at 0x7f9b81e62e80> JSON: {
  "_type": "CatXL",
  "attachment": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 3000000.0,
    "value_date": null
  },
  "created": "2022-09-05T23:20:01.212052+00:00",
  "description": "Test Layer for Python Training",
  "expiry_date": "2023-01-01T00:00:00+00:00",
  "fees": [],
  "franchise": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 0.0,
    "value_date": null
  },
  "id": "7f825da0-d02b-4494-9fae-853fcfc7a328",
  "inception_date": "2022-01-01T00:00:00+00:00",
  "limit": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 10000000.0,
    "value_date": null
  },
  "loss_sets": [
    {
      "ref_id": "5228a1eb-0546-4768-95c1-cac600202a53"
    }
  ],
  "meta_data": {
    "Test-Latest": true,
    "Test-LayerId": 12345,
    "Test-Purpose": "Training",
    "Test-Version": 1
  },
  "modified": "2022-09-06T09:

In [ ]:
# Create another layer
second_layer = Layer(
    # Type of layer: CatXL
    type="CatXL",
    # Loss sets associated with the layer
    loss_sets=[ls],
    # 3M USD attachment
    attachment=MonetaryUnit(13e6, "USD"),
    # 10M USD limit
    limit=MonetaryUnit(5e6, "USD"),
    # Franchise deductible is a required field, but we set it to 0 here.
    franchise=MonetaryUnit(0, "USD"),
    # The reinsurer's participation: 25%
    participation=0.10,
    # Inception date (optional, inclusive, supports `datetime`)
    inception_date="2022-01-01T00:00:00Z",
    # Expiry date (optional, exclusive, supports `datetime`)
    expiry_date="2023-01-01T00:00:00Z",
    # Description (optional)
    description="Test Layer 2 for Python Training",
    # Additional metadata fields (optional)
    meta_data={
        "Test-LayerId": 12346,
        "Test-Purpose": "Training",
        "Test-Version": 1,
        "Test-Latest": True,
    }
).save()

In [ ]:
# Add second layer to portfolio
portfolio = Portfolio.retrieve(portfolio.id)
portfolio.layers.append(second_layer)
portfolio.save()

<Portfolio id=5714a745-4809-4665-86d2-29f1ebba1d91 at 0x7f9b81f13070> JSON: {
  "_type": "StaticPortfolio",
  "created": "2022-09-05T23:38:22.122976+00:00",
  "description": "Test Portfolio for Python Training",
  "id": "5714a745-4809-4665-86d2-29f1ebba1d91",
  "layers": [
    {
      "ref_id": "7f825da0-d02b-4494-9fae-853fcfc7a328"
    },
    {
      "ref_id": "917c6a08-1006-4d6c-99b1-d992ba869916"
    }
  ],
  "meta_data": {
    "Test-Latest": true,
    "Test-PortfolioId": 12345,
    "Test-Purpose": "Training",
    "Test-Version": 1
  },
  "modified": "2022-09-06T10:02:00.823330+00:00",
  "name": "Test Portfolio for Python Training"
}

## Creating Views

Views are one of the Prime platform's most important features. They are used to associate a financial structure, such as a Layer or Portfolio, and their respectively associated Loss Sets, with an Analysis Profile, to facilitate the calculation of metrics. Views are immutable objects that cannot be modified once they are created. Upon creation each view is assigned an ID which is distinct for data that encompasses the view. Two identical views, which produce identical metrics, are assigned identical IDs.

In order to create a Layer View we require a Layer object and an Analysis Profile. For this example, we will use the modified layer above `layer` and retrieve an appropriate Analysis Profile.

In [ ]:
from analyzere import LayerView

In [ ]:
lv = LayerView(
    analysis_profile=AnalysisProfile.retrieve("f77f888a-138a-45aa-a3bd-8e5fd66151fc"),
    layer=layer
).save()

In [ ]:
lv

<LayerView id=f96f9f1a-3a46-ee00-ff76-203c84acea02 at 0x7f9b82edfd30> JSON: {
  "analysis_profile": {
    "ref_id": "f77f888a-138a-45aa-a3bd-8e5fd66151fc"
  },
  "id": "f96f9f1a-3a46-ee00-ff76-203c84acea02",
  "layer": {
    "_type": "CatXL",
    "attachment": {
      "currency": "USD",
      "rate": null,
      "rate_currency": null,
      "value": 3000000.0,
      "value_date": null
    },
    "description": "Test Layer for Python Training",
    "expiry_date": "2023-01-01T00:00:00+00:00",
    "fees": [],
    "franchise": {
      "currency": "USD",
      "rate": null,
      "rate_currency": null,
      "value": 0.0,
      "value_date": null
    },
    "inception_date": "2022-01-01T00:00:00+00:00",
    "limit": {
      "currency": "USD",
      "rate": null,
      "rate_currency": null,
      "value": 10000000.0,
      "value_date": null
    },
    "loss_sets": [
      {
        "ref_id": "5228a1eb-0546-4768-95c1-cac600202a53"
      }
    ],
    "meta_data": {
      "Test-Latest": true,

Similarly, in order to create a Portfolio View of a portfolio, we require a portfolio object `portfolio` and an appropriate Analysis Profile.

In [ ]:
from analyzere import PortfolioView

In [ ]:
pv = PortfolioView(
    analysis_profile=AnalysisProfile.retrieve("f77f888a-138a-45aa-a3bd-8e5fd66151fc"),
    portfolio=portfolio,
    target_currency="USD",
).save()

In [ ]:
pv
# Note: Creating the Portfolio View has implicitly created Layer Views 
# for all of my layers in the portfolio. Also note that the Layer View ID 
# for my first layer is identical to the Layer View ID created above because 
# they refer to the same data.

<PortfolioView id=9deebffa-5702-c59e-871e-46c7569d050c at 0x7f9b8200e070> JSON: {
  "analysis_profile": {
    "ref_id": "f77f888a-138a-45aa-a3bd-8e5fd66151fc"
  },
  "id": "9deebffa-5702-c59e-871e-46c7569d050c",
  "layer_views": [
    {
      "ref_id": "f96f9f1a-3a46-ee00-ff76-203c84acea02"
    },
    {
      "ref_id": "3fae7446-7796-51ea-9862-c1492432557e"
    }
  ],
  "portfolio": {
    "ref_id": "5714a745-4809-4665-86d2-29f1ebba1d91"
  },
  "premium": {
    "currency": "USD",
    "rate": null,
    "rate_currency": null,
    "value": 250000.0,
    "value_date": null
  },
  "target_currency": "USD",
  "ylt_id": "4f27605d-0bc9-09b0-d61a-1109701839a1"
}

It is also possible to change the default reporting currency for a Layer View or Portfolio View by specifying the `target_currency` attribute during construction.

In [ ]:
pv_gbp = PortfolioView(
    analysis_profile=AnalysisProfile.retrieve("f77f888a-138a-45aa-a3bd-8e5fd66151fc"),
    portfolio=portfolio,
    target_currency="GBP",
).save()

In [ ]:
pv_gbp

<PortfolioView id=197cb34e-603b-c062-0eb3-cc6f343278ff at 0x7f9b82edfa00> JSON: {
  "analysis_profile": {
    "ref_id": "f77f888a-138a-45aa-a3bd-8e5fd66151fc"
  },
  "id": "197cb34e-603b-c062-0eb3-cc6f343278ff",
  "layer_views": [
    {
      "ref_id": "f96f9f1a-3a46-ee00-ff76-203c84acea02"
    },
    {
      "ref_id": "3fae7446-7796-51ea-9862-c1492432557e"
    }
  ],
  "portfolio": {
    "ref_id": "5714a745-4809-4665-86d2-29f1ebba1d91"
  },
  "premium": {
    "currency": "GBP",
    "rate": null,
    "rate_currency": null,
    "value": 184571.31466456008,
    "value_date": null
  },
  "target_currency": "GBP",
  "ylt_id": "07d013c3-0a3b-62b7-e7fc-3d9ef63d826b"
}

## Querying metrics

As indicated above, views are primarily used to obtain metrics for a specific Layer or Portfolio using the modelling configuration specified in the Analysis Profile. The Prime platform offers the ability to query a number of different risk metrics through a simple yet powerful interface. Some common risk metrics such as Expected Loss (AAL) have short-cut functions, but the unterlying calculations are identical.

In [ ]:
# Query various expected losses (AAL) from a Layer View
print("Expected loss (losses-only, no filters): ", lv.el())
print("Expected loss (losses-only, US Florida WS): ", lv.el(filter="US_Florida_WS"))
print(
    "Expected loss (net Prem/ReinPrem, no filters): ",
    lv.el(perspective="NetLoss,Premium,ReinstatementPremium")
)
print(
    "Expected loss (net ReinPrem, US Florida WS): ",
    lv.el(perspective="NetLoss,ReinstatementPremium", filter="US_Florida_WS")
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    lv.el(filter="US_Florida_WS", aggregation_method="OEP")
)

Expected loss (losses-only, no filters):  1201231.8999593004
Expected loss (losses-only, US Florida WS):  442970.5087417
Expected loss (net Prem/ReinPrem, no filters):  160968.45448427505
Expected loss (net ReinPrem, US Florida WS):  429541.39932938863
Expected loss OEP (losses-only, US Florida WS):  126192.92028870006


In [ ]:
# Note that for stand-alone metrics participation is not applied by default.
print("Our Expected loss (losses-only, no filters): ", lv.el(apply_participation=True))
print(
    "Our Expected loss (losses-only, US Florida WS): ", 
    lv.el(apply_participation=True, filter="US_Florida_WS")
)
print(
    "Our Expected loss (net Prem/ReinPrem, no filters): ",
    lv.el(
        apply_participation=True,
        perspective="NetLoss,Premium,ReinstatementPremium"
    )
)
print(
    "Our Expected loss (net ReinPrem, US Florida WS): ",
    lv.el(
        perspective="NetLoss,ReinstatementPremium", 
        filter="US_Florida_WS",
        apply_participation=True
    )
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    lv.el(
        filter="US_Florida_WS", 
        aggregation_method="OEP",
        apply_participation=True
    )
)

Our Expected loss (losses-only, no filters):  300307.9749898251
Our Expected loss (losses-only, US Florida WS):  110742.627185425
Our Expected loss (net Prem/ReinPrem, no filters):  40242.11362106876
Our Expected loss (net ReinPrem, US Florida WS):  107385.34983234716
Expected loss OEP (losses-only, US Florida WS):  31548.230072175014


In [ ]:
# Similarly we can query the expected losses (AAL) for Portfolio Views
print("Expected loss (losses-only, no filters): ", pv.el())
print("Expected loss (losses-only, US Florida WS): ", pv.el(filter="US_Florida_WS"))
print(
    "Expected loss (net Prem/ReinPrem, no filters): ",
    pv.el(perspective="NetLoss,Premium,ReinstatementPremium")
)
print(
    "Expected loss (net ReinPrem, US Florida WS): ",
    pv.el(perspective="NetLoss,ReinstatementPremium", filter="US_Florida_WS")
)
print(
    "Expected loss OEP (losses-only, US Florida WS): ", 
    pv.el(
        filter="US_Florida_WS", 
        aggregation_method="OEP",
    )
)
# Note that for portfolio view metrics individual layer participations are always applied.

Expected loss (losses-only, no filters):  306415.5202477251
Expected loss (losses-only, US Florida WS):  112918.73285072493
Expected loss (net Prem/ReinPrem, no filters):  46349.65887896877
Expected loss (net ReinPrem, US Florida WS):  109561.45549764714
Expected loss OEP (losses-only, US Florida WS):  112786.37973722494


### VaR / TVaR / CTE

In [ ]:
# Tail Metrics are various statistics that are computed for the tail of the loss distribution
lv.tail_metrics(0.1)

<EmbeddedResource at 0x7f9b81e98e50> JSON: {
  "_type": "LayerViewMetrics",
  "context": {
    "_type": "LayerViewMetricsContext",
    "aggregation_method": "AEP",
    "apply_participation": false,
    "currency": "USD",
    "filter": "World Wide All Perils",
    "perspective": "LossNetOfAggregateTerms",
    "probability": 0.1,
    "secondary_uncertainty": true
  },
  "kurtosis": -1.0104069132413316,
  "max": 30000000.0,
  "mean": 11955209.30187401,
  "min": 829597.3980000005,
  "skewness": 0.6963959570232366,
  "variance": 108165080701610.6
}

In [ ]:
# Layer View tail metrics
return_period = 10
probability = 1.0 / return_period
print(
    f"Layer 1-in-{return_period} CTE (losses-only, no filter):",
    lv.tail_metrics(probability).mean
)
print(
    f"Layer 1-in-{return_period} CTE (losses-only, US Florida WS):",
    lv.tail_metrics(probability, filter="US_Florida_WS").mean
)
print(
    f"Layer 1-in-{return_period} CTE (net Prem/ReinPrem, no filter):",
    lv.tail_metrics(probability, perspective="NetLoss,Premium,ReinstatementPremium").mean
)
print(
    f"Layer 1-in-{return_period} CTE (net ReinPrem, US Florida WS):",
    lv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS"
    ).mean
)
print(
    f"Layer 1-in-{return_period} OEP CTE (net ReinPrem, US Florida WS):",
    lv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS",
        aggregation_method="OEP"
    ).mean
)

Layer 1-in-10 CTE (losses-only, no filter): 11955209.30187401
Layer 1-in-10 CTE (losses-only, US Florida WS): 4429705.087417
Layer 1-in-10 CTE (net Prem/ReinPrem, no filter): 10555430.332009684
Layer 1-in-10 CTE (net ReinPrem, US Florida WS): 4295413.993293886
Layer 1-in-10 OEP CTE (net ReinPrem, US Florida WS): 1246360.1194352317


In [ ]:
# Portfolio View tail metrics (participation applied)
return_period = 10
probability = 1.0 / return_period
print(
    f"Portfolio 1-in-{return_period} CTE (losses-only, no filter):",
    pv.tail_metrics(probability).mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (losses-only, US Florida WS):",
    pv.tail_metrics(probability, filter="US_Florida_WS").mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (net Prem/ReinPrem, no filter):",
    pv.tail_metrics(probability, perspective="NetLoss,Premium,ReinstatementPremium").mean
)
print(
    f"Portfolio 1-in-{return_period} CTE (net ReinPrem, US Florida WS):",
    pv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS"
    ).mean
)
print(
    f"Portfolio 1-in-{return_period} OEP CTE (net ReinPrem, US Florida WS):",
    pv.tail_metrics(
        probability, 
        perspective="NetLoss,ReinstatementPremium",
        filter="US_Florida_WS",
        aggregation_method="OEP"
    ).mean
)

Portfolio 1-in-10 CTE (losses-only, no filter): 3049877.778047502
Portfolio 1-in-10 CTE (losses-only, US Florida WS): 1129187.3285072492
Portfolio 1-in-10 CTE (net Prem/ReinPrem, no filter): 2699933.035581421
Portfolio 1-in-10 CTE (net ReinPrem, US Florida WS): 1095614.5549764715
Portfolio 1-in-10 OEP CTE (net ReinPrem, US Florida WS): 1094357.2003982218


### Exceedance Probability

The probability a loss reaches or exceeds a certain threshold. Used to determine attachment, exhaustion, and reinstatement probabilities.

In [ ]:
print(f"Layer attachment probability:", lv.ep(0.0).probability)
print(
    f"Layer occurrence exhaustion probability:", 
    lv.ep(
        lv.layer.limit.value, 
        inclusive_threshold=True,
        aggregation_method="OEP",
    ).probability
)
print(
    f"Layer aggregate exhaustion probability:", 
    lv.ep(
        lv.layer.limit.value*(len(lv.layer.reinstatements)+1), 
        inclusive_threshold=True
    ).probability
)
print(
    f"Layer reinstatement probability:", 
    lv.ep(
        lv.layer.limit.value, 
        inclusive_threshold=False,
    ).probability
)

Layer attachment probability: 0.115
Layer occurrence exhaustion probability: 0.0133
Layer aggregate exhaustion probability: 0.0162
Layer reinstatement probability: 0.0426


## Performance Optimizations

HTTP REST requests incur some overhead through latency between the client and server, and processing overhead on the server side. Overall application throughput can be greatly enhanced by employing two techniques:

1. Vectorization: to reduce that overhead
2. Parallelism: to amortize the overhead

### Vectorization

Most metrics request offer the opportunity to specify a vector of probabilities or thresholds which bundle multiple requests into a single request.

In [ ]:
[ point.mean for point in lv.tail_metrics([1.0, 0.5, 0.2, 0.1]) ]

[1201231.8999593004, 2402463.799918601, 6006159.499796502, 11955209.30187401]

In [ ]:
[ 
    threshold.probability 
    for threshold in lv.ep([0.5e6, 1e6, 2e5, 3e6], inclusive_threshold=True) 
]

[0.1053, 0.0972, 0.1102, 0.0738]

### Parallelism

Some parameters to metrics requests cannot be vectorized. For example, requesting metrics for different filters are always separate request. However, sometimes metrics for a particularly large number of filters and/or perspectives must be retrieved. In those situations it is beneficial to issue multiple requests to the platform concurrenly to amortize each individual request's overhead across parallel requests. For example:

In [ ]:
filters = [
    "Caribbean_WS",
    "Central_America_WS",
    "Mexico_WS",
    "US_Florida_WS",
    "US_Gulf_WS",
    "US_Mid_Atlantic_WS",
    "US_North_East_WS",
    "US_Texas_WS"
]
aggregation_methods = ["AEP", "OEP"]
perspectives = ["NetLoss", "NetLoss,ReinstatementPremium"]

In [ ]:
from itertools import product
requests = list(product(filters, aggregation_methods, perspectives))
print(f"Number of requests required: {len(requests)}")

Number of requests required: 32


In [ ]:
# Execute multiple requests in parallel using a ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor as Pool

with Pool(8) as pool:
    
    # Helper function
    def get_cte(filter, aggregation_method, perspective):
        return lv.tail_metrics(
            0.1, 
            filter=filter, 
            aggregation_method=aggregation_method, 
            perspective=perspective
        ).mean
    
    results = list(pool.map(lambda job: get_cte(*job), requests))

list(zip(requests, results))

[(('Caribbean_WS', 'AEP', 'NetLoss'), 2756913.486341002),
 (('Caribbean_WS', 'AEP', 'NetLoss,ReinstatementPremium'), 2649203.238704228),
 (('Caribbean_WS', 'OEP', 'NetLoss'), 1313666.0684949988),
 (('Caribbean_WS', 'OEP', 'NetLoss,ReinstatementPremium'), 1265761.8511329505),
 (('Central_America_WS', 'AEP', 'NetLoss'), 0.0),
 (('Central_America_WS', 'AEP', 'NetLoss,ReinstatementPremium'), 0.0),
 (('Central_America_WS', 'OEP', 'NetLoss'), 0.0),
 (('Central_America_WS', 'OEP', 'NetLoss,ReinstatementPremium'), 0.0),
 (('Mexico_WS', 'AEP', 'NetLoss'), 0.0),
 (('Mexico_WS', 'AEP', 'NetLoss,ReinstatementPremium'), 0.0),
 (('Mexico_WS', 'OEP', 'NetLoss'), 0.0),
 (('Mexico_WS', 'OEP', 'NetLoss,ReinstatementPremium'), 0.0),
 (('US_Florida_WS', 'AEP', 'NetLoss'), 4429705.087417),
 (('US_Florida_WS', 'AEP', 'NetLoss,ReinstatementPremium'), 4295413.993293886),
 (('US_Florida_WS', 'OEP', 'NetLoss'), 1261929.2028870005),
 (('US_Florida_WS', 'OEP', 'NetLoss,ReinstatementPremium'),
  1246360.1194352317

## Advanced Metrics

### Co-Metrics

In [ ]:
# Contribution of Layer 2 to the Portfolio 1-in-10
pv.co_metrics(
    0.1, 
    component_id=pv.layer_views[1].id, 
    component_type="LayerView", 
    include_primary_metrics=True
)

<EmbeddedResource at 0x7f9b82391400> JSON: {
  "_type": "PortfolioViewCoMetrics",
  "component_metrics": {
    "correlation": 0.5858587446200116,
    "covariance": 2532214039313.074,
    "mean": 610754.5257900001,
    "min": 0.0
  },
  "context": {
    "_type": "PortfolioViewCoMetricsContext",
    "aggregation_method": "AEP",
    "apply_participation": true,
    "component": {
      "ref_id": "3fae7446-7796-51ea-9862-c1492432557e"
    },
    "component_filter": "World Wide All Perils",
    "component_perspective": "LossNetOfAggregateTerms",
    "currency": "USD",
    "filter": "World Wide All Perils",
    "include_primary_metrics": true,
    "perspective": "LossNetOfAggregateTerms",
    "probability": 0.1,
    "secondary_uncertainty": true
  },
  "primary_metrics": {
    "kurtosis": -0.940667252123986,
    "max": 8000000.0,
    "mean": 3049877.778047502,
    "min": 207399.34950000013,
    "skewness": 0.7309804551067125,
    "variance": 7240960404424.517
  }
}

In [ ]:
# Contribution of Layer 2 US_Florida_WS to the Portfolio 1-in-10
pv.co_metrics(
    0.1, 
    component_id=pv.layer_views[1].id, 
    component_type="LayerView",
    component_filter="US_Florida_WS",
    include_primary_metrics=True
).component_metrics.mean

217610.56653000004

### Window Metrics

In [ ]:
# 1-in-5 good vs. 1-in-5 bad years
pv.window_metrics([(0.8, 1.0), (0.0, 0.2)])

[<EmbeddedResource at 0x7f9bb76cc760> JSON: {
   "_type": "PortfolioViewMetrics",
   "context": {
     "_type": "PortfolioViewMetricsContext",
     "aggregation_method": "AEP",
     "apply_participation": true,
     "currency": "USD",
     "filter": "World Wide All Perils",
     "max_probability": 1.0,
     "min_probability": 0.8,
     "perspective": "LossNetOfAggregateTerms",
     "secondary_uncertainty": true
   },
   "kurtosis": 0.0,
   "max": 0.0,
   "mean": 0.0,
   "min": 0.0,
   "skewness": 0.0,
   "variance": 0.0
 },
 <EmbeddedResource at 0x7f9bb76cc430> JSON: {
   "_type": "PortfolioViewMetrics",
   "context": {
     "_type": "PortfolioViewMetricsContext",
     "aggregation_method": "AEP",
     "apply_participation": true,
     "currency": "USD",
     "filter": "World Wide All Perils",
     "max_probability": 0.2,
     "min_probability": 0.0,
     "perspective": "LossNetOfAggregateTerms",
     "secondary_uncertainty": true
   },
   "kurtosis": 1.3453308660836827,
   "max": 8000